In [33]:
import re
import nltk
import yake 
import time
import pandas as pd
from summa import keywords as sk
from keybert import KeyBERT
from googlesearch import search
from bs4 import BeautifulSoup
from urllib.request import urlopen
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
kw_model = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')

stop_words = set(stopwords.words('english') + stop_word)
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

tag_re = re.compile(r'<[^>]+>')

[nltk_data] Downloading package wordnet to C:\Users\Shashank
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Shashank
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [34]:
def scrapper(query_string, num_of_pages, stop_at):
    """Scrapes search results for the provided query string."""
    documents = []
    for search_result in search(query_string, tld="com", num=num_of_pages, stop=stop_at, pause=2):
        try:
            html_content = urlopen(search_result).read()
            soup = BeautifulSoup(html_content, features="html.parser")
            documents.append(soup.body)
        except:
            pass
    return documents

def cleaner(text):
    """Cleans the provided text by removing tags, newlines, emails, symbols, etc."""
    text = tag_re.sub('', str(text))
    text = text.replace(r'\n', ' ').replace('\n', '')
    text = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', ' ', text)
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = re.sub(r"(^|\W)\d+", ' ', text)
    text = lemmatizer.lemmatize(text)
    text = ps.stem(text)
    return text

def keyword_extractor(method, corpus, top_n, max_ngram_size):
    """Extracts keywords from the corpus using the specified method."""
    try:
        if method == 'keybert':
            return [word[0] for word in kw_model.extract_keywords(corpus, keyphrase_ngram_range=(1, max_ngram_size), stop_words='english', top_n=top_n)]

        if method == 'text_rank':
            return [word[0] for word in sk.keywords(corpus, scores=True)[:top_n]]

        if method == 'yake':
            kw_yake = yake.KeywordExtractor(top=top_n, stopwords=stop_words, n=max_ngram_size)
            return [word[0] for word in kw_yake.extract_keywords(corpus)]

    except:
        print('Error: Keyword extraction method not found!')
        return []

def driver(query_string, num_of_pages, stop_at, top_n_words, grams, method):
    """Main driver function to extract and aggregate keywords."""
    data = []
    for query in query_string:
        documents = scrapper(query, num_of_pages, stop_at)
        keywords = [word for doc in documents for word in keyword_extractor(method, cleaner(doc), top_n_words, grams)]
        keywords = [k for k in keywords if set(k.split(' ')).isdisjoint(stop_words)]
        
        keyword_counts = [[keyword, keywords.count(keyword)] for keyword in set(keywords)]
        keyword_freq_df = pd.DataFrame(keyword_counts, columns=['Keyword', 'Freq']).sort_values(by='Freq', ascending=False).reset_index(drop=True)
        keyword_freq_df = keyword_freq_df[keyword_freq_df["Freq"] > 3]
        keyword_freq_df['Google Search Query'] = query
        
        data.append(keyword_freq_df)

    final_keywords_df = pd.concat(data)
    final_keywords_df.to_json('keywords.json', orient='records', lines=True)

In [35]:
start_time = time.time()
driver(query_string = ['Data Mining', 'Machine Learning'], num_of_pages = 10, stop_at = 10, top_n_words = 100, grams = 3, method = 'yake')
print("--- %s seconds ---" % (time.time() - start_time))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


--- 55.909438610076904 seconds ---


In [4]:
#keyword_extractor('keybert', clean[0], 10)
#keyword_extractor('text_rank', clean[doc], 50)